In [1]:
#in this notebook we clean and analyse the data as first step
#Load data from SQLite database
#Convert date to datetime
#Drop time column bc empty
#Split score into home_goals and away_goals
#Save as new matches_clean.csv


In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("/home/valentina/RandI/project/la-quiniela/laliga.sqlite")

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tables)


matches = pd.read_sql("SELECT * FROM Matches LIMIT 10", conn)
matches.head(20)


          name
0  Predictions
1      Matches


,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
5,1928-1929,1,2,2/17/29,None,Barcelona,Real Madrid,1:2
6,1928-1929,1,2,2/17/29,None,Athletic,Espanyol,9:0
7,1928-1929,1,2,2/17/29,None,Athletic Madrid,Donostia,0:3
8,1928-1929,1,2,2/17/29,None,Real Unión,Racing,3:1
9,1928-1929,1,2,2/17/29,None,Catalunya,Arenas Club,5:2


In [3]:
matches['date'] = pd.to_datetime(matches['date'], format='%m/%d/%y', errors='coerce')
# Fix century issue if needed
matches.loc[matches['date'].dt.year > 2025, 'date'] -= pd.DateOffset(years=100)

matches['date'].head(10)

0   1929-02-10
1   1929-02-10
2   1929-02-10
3   1929-02-10
4   1929-02-12
5   1929-02-17
6   1929-02-17
7   1929-02-17
8   1929-02-17
9   1929-02-17
Name: date, dtype: datetime64[ns]

In [4]:
def parse_score(s):
    if not isinstance(s, str): 
        return pd.Series([None, None])
    if ':' not in s:
        return pd.Series([None, None])
    h, a = s.split(':')
    try:
        return pd.Series([int(h), int(a)])
    except ValueError:
        return pd.Series([None, None])

matches[['home_goals', 'away_goals']] = matches['score'].apply(parse_score)


In [5]:
def result_label(row):
    if pd.isna(row['home_goals']) or pd.isna(row['away_goals']):
        return None
    if row['home_goals'] > row['away_goals']:
        return '1'  # home win
    elif row['home_goals'] < row['away_goals']:
        return '2'  # away win
    else:
        return 'X'  # draw

matches['result'] = matches.apply(result_label, axis=1)


In [6]:
matches[['season','home_team','away_team','score','result']].head(10)


,season,home_team,away_team,score,result
0,1928-1929,Arenas Club,Athletic Madrid,2:3,2
1,1928-1929,Espanyol,Real Unión,3:2,1
2,1928-1929,Real Madrid,Catalunya,5:0,1
3,1928-1929,Donostia,Athletic,1:1,X
4,1928-1929,Racing,Barcelona,0:2,2
5,1928-1929,Barcelona,Real Madrid,1:2,2
6,1928-1929,Athletic,Espanyol,9:0,1
7,1928-1929,Athletic Madrid,Donostia,0:3,2
8,1928-1929,Real Unión,Racing,3:1,1
9,1928-1929,Catalunya,Arenas Club,5:2,1


In [7]:
matches.info()
matches['result'].value_counts(dropna=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   season      10 non-null     object        
 1   division    10 non-null     int64         
 2   matchday    10 non-null     int64         
 3   date        10 non-null     datetime64[ns]
 4   time        0 non-null      object        
 5   home_team   10 non-null     object        
 6   away_team   10 non-null     object        
 7   score       10 non-null     object        
 8   home_goals  10 non-null     int64         
 9   away_goals  10 non-null     int64         
 10  result      10 non-null     object        
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 1012.0+ bytes


result
1    5
2    4
X    1
Name: count, dtype: int64

In [9]:
matches = pd.read_sql("SELECT * FROM Matches", conn)

# Drop the empty column
matches_clean = matches.drop(columns=["time"])
matches_clean.info()
# Save cleaned version
matches_clean.to_csv("matches_clean.csv", index=False)
df = pd.read_csv("matches_clean.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48780 entries, 0 to 48779
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   season     48780 non-null  object
 1   division   48780 non-null  int64 
 2   matchday   48780 non-null  int64 
 3   date       48780 non-null  object
 4   home_team  48780 non-null  object
 5   away_team  48780 non-null  object
 6   score      48000 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.6+ MB
0    2/10/29
1    2/10/29
2    2/10/29
3    2/10/29
4    2/12/29
Name: date, dtype: object


In [10]:
import pandas as pd

# 1️⃣ Load the dataset
df = pd.read_csv("matches_clean.csv")

# 2️⃣ Remove completely empty rows
df = df.dropna(how="all")

# 3️⃣ Parse dates (day/month/two-digit-year format)
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%y", errors="coerce")

# 4️⃣ Fix incorrect century interpretation (e.g. 2029 → 1929)
df.loc[df["date"].dt.year > 2025, "date"] -= pd.offsets.DateOffset(years=100)

# 5️⃣ Drop rows where date couldn't be parsed
df = df.dropna(subset=["date"])

# 6️⃣ Quick check
print(df["date"].head(10))
print("Earliest date:", df["date"].min())
print("Latest date:", df["date"].max())
print("Data shape after cleaning:", df.shape)

# 7️⃣ Save the cleaned file permanently in consistent ISO format
df.to_csv("matches_clean.csv", index=False, date_format="%Y-%m-%d")
print("✅ Cleaned data saved to matches_clean.csv")


0    1929-10-02
1    1929-10-02
2    1929-10-02
3    1929-10-02
4    1929-12-02
15   1929-03-03
16   1929-03-03
17   1929-03-03
18   1929-03-03
19   1929-03-03
Name: date, dtype: datetime64[ns]
Earliest date: 1929-01-12 00:00:00
Latest date: 2022-11-05 00:00:00
Data shape after cleaning: (19686, 7)
✅ Cleaned data saved to matches_clean.csv


In [11]:

print(df["date"].head())
print(df["date"].dtype)
print(df["date"].isna().sum())


0   1929-10-02
1   1929-10-02
2   1929-10-02
3   1929-10-02
4   1929-12-02
Name: date, dtype: datetime64[ns]
datetime64[ns]
0
